# Firewall Log Analysis

In [1]:
import json
import pandas as pd

# Try line-delimited JSON first, then fall back to a single JSON object/array.
try:
    df = pd.read_json("lab_2_firewall_log.json", lines=True)
except ValueError:
    with open("lab_2_firewall_log.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    df = pd.DataFrame(data)

df

,@timestamp,cisco.asa.destination_interface,cisco.asa.message_id,cisco.asa.rule_name,cisco.asa.source_interface,destination.address,destination.ip,destination.port,event.action,event.category,...,log.syslog.priority,log.syslog.severity.code,timestamp,src_ip,dest_ip,src_port,dest_port,protocol,action,bytes
0,2013-04-15T09:36:50.000-02:00,outside,106023,acl_dmz,dmz,192.0.0.8,192.0.0.8,53.0,firewall-rule,[network],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-04-15T09:36:50.000-02:00,outside,106023,acl_dmz,dmz,192.0.0.8,192.0.0.8,53.0,firewall-rule,[network],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-04-15T13:34:34.000Z,outside,106100,acl_in,inside,192.0.0.89,,2000.0,firewall-rule,[network],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-04-24T16:00:28.000-02:00,outside,106100,inside,inside,192.0.2.10,192.0.2.10,53.0,firewall-rule,[network],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-04-24T16:00:27.000-02:00,outside,106100,inside,inside,192.0.2.57,192.0.2.57,53.0,firewall-rule,[network],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2026-02-02T20:23:49.974906Z,10.0.0.5,172.217.3.110,51184.0,443.0,TCP,ALLOW,93885.0
135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2026-02-02T20:24:49.974906Z,10.0.0.5,172.217.3.110,55418.0,443.0,TCP,ALLOW,100204.0
136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2026-02-02T20:25:49.974906Z,10.0.0.5,172.217.3.110,43717.0,443.0,TCP,ALLOW,98256.0
137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2026-02-02T20:26:49.974906Z,10.0.0.5,172.217.3.110,50093.0,443.0,TCP,ALLOW,104513.0


In [2]:
df.head()

,@timestamp,cisco.asa.destination_interface,cisco.asa.message_id,cisco.asa.rule_name,cisco.asa.source_interface,destination.address,destination.ip,destination.port,event.action,event.category,...,log.syslog.priority,log.syslog.severity.code,timestamp,src_ip,dest_ip,src_port,dest_port,protocol,action,bytes
0,2013-04-15T09:36:50.000-02:00,outside,106023,acl_dmz,dmz,192.0.0.8,192.0.0.8,53.0,firewall-rule,[network],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-04-15T09:36:50.000-02:00,outside,106023,acl_dmz,dmz,192.0.0.8,192.0.0.8,53.0,firewall-rule,[network],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-04-15T13:34:34.000Z,outside,106100,acl_in,inside,192.0.0.89,,2000.0,firewall-rule,[network],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-04-24T16:00:28.000-02:00,outside,106100,inside,inside,192.0.2.10,192.0.2.10,53.0,firewall-rule,[network],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-04-24T16:00:27.000-02:00,outside,106100,inside,inside,192.0.2.57,192.0.2.57,53.0,firewall-rule,[network],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Columns: 115 entries, @timestamp to bytes
dtypes: float64(24), object(91)
memory usage: 125.0+ KB


In [4]:
# Basic frequency table for a likely column if present.
candidate_cols = ["action", "protocol", "src_ip", "dst_ip", "src_port", "dst_port"]
available = [c for c in candidate_cols if c in df.columns]
if available:
    df[available[0]].value_counts().head(20)
else:
    df.columns